# Möbius‑Digit‑Thread Scanner (clean rebuild)
**gap_scale 20, spacing 3, k = 4**

This is the exact parameter combo likely to push the miss back under 1 %.

In [1]:
import mpmath as mp, pandas as pd, itertools, re
mp.mp.dps=260
ALPHA_INV=mp.mpf('137.035999084'); ALPHA=1/ALPHA_INV; PHI=(1+mp.sqrt(5))/2

In [2]:
def digits_of(num,n=700):
    s=str(num).replace('.','')[1:]
    return (s+'0'*n)[:n]
CONST_BANK={n:digits_of(v) for n,v in {
 'pi':mp.pi,'phi':PHI,'e':mp.e,'gamma':mp.euler,
 'delta':mp.mpf('4.669201609'),'sqrt2':mp.sqrt(2)}.items()}

In [3]:
def primes_upto(n):
    s=[True]*(n+1); s[0:2]=[False,False]
    for i in range(2,int(n**0.5)+1):
        if s[i]: s[i*i:n+1:i]=[False]*len(range(i*i,n+1,i))
    return [i for i,v in enumerate(s) if v]
def prime_gaps(cnt):
    ps=primes_upto(3000);return [ps[i+1]-ps[i] for i in range(cnt)]

In [4]:
def build_tail(bl=400,sp=3,gs=20):
    gaps=[round(g/gs,4) for g in prime_gaps(bl//sp)]
    tail=[]; gi=iter(gaps)
    for i in range(bl):
        tail.append(1)
        if (i+1)%sp==0: tail.append(next(gi,1))
    return tail
def cf_to_real(co):
    v=mp.mpf(0)
    for a in reversed(co): v=1/(a+v)
    return v

In [5]:
def breath(x,tiny,phi):
    x=phi*PHI/x; x-=tiny; x=1/x/9; x=1/x; return x/9

In [6]:
powers=[9,27,81,127,729]
def cthread(xdig):
    for nm,b in CONST_BANK.items():
        for L in range(8,3,-1):
            for i in range(len(xdig)-L+1):
                sub=xdig[i:i+L]
                if sub in b: return f'{nm}:{sub}'
    return '-'

## Run scan

In [7]:
tail=build_tail()
seed=cf_to_real(tail)
tiny=ALPHA_INV/1e4
phi_c=0.003944
phi_vals=[phi_c+i*1e-7 for i in range(-5,6)]
rows=[]
for bits in (64,128):
    for phi in phi_vals:
        x=breath(seed+tiny,tiny,phi)
        rel=float(abs((x-ALPHA)/ALPHA))
        rows.append(dict(bits=bits,phi=phi,rel_err=rel,thread=cthread(str(x)[2:402])))
pd.DataFrame(rows).sort_values('rel_err').head(40)

,bits,phi,rel_err,thread
10,64,0.003944,1.407889,pi:0955
21,128,0.003944,1.407889,pi:0955
20,128,0.003944,1.407926,pi:59230
9,64,0.003944,1.407926,pi:59230
8,64,0.003944,1.407963,pi:56482
19,128,0.003944,1.407963,pi:56482
18,128,0.003944,1.408001,pi:3993
7,64,0.003944,1.408001,pi:3993
6,64,0.003944,1.408038,pi:3383
17,128,0.003944,1.408038,pi:3383
